# Data Preparation

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display



loan_dev = pd.read_csv('data/loan_dev.csv', dtype={'date':object})
account = pd.read_csv('data/account.csv', dtype={'date':object})
card_dev = pd.read_csv('data/card_dev.csv', dtype={'issued':object})
client = pd.read_csv('data/client.csv')
disposition = pd.read_csv('data/disp.csv')
district = pd.read_csv('data/district.csv')
trans_dev = pd.read_csv('data/trans_dev.csv', dtype={'date':object})

pd.set_option('display.max_columns', None)


## Loan preparation

### Save

In [ ]:
loan_dev.rename(columns={'date':'loan_date', 'amount': 'loan_amount', 'duration': 'loan_duration', 'payments': 'loan_payments' }, inplace=True)
loan_dev.to_csv('data_processed/loan_dev.csv', index=False)
loan_dev

## Account preparation

In [ ]:
account.drop(['date'], inplace=True, axis=1)

### Save

In [ ]:
account.rename(columns={'frequency':'account_frequency', 'district_id': 'account_district_id'}, inplace=True)
account.to_csv('data_processed/account.csv', index=False)
account

## Card preparation

There are only a few loans associated with a credit card

In [ ]:
card_dev.drop(["issued"], inplace=True, axis=1)
card_dev = card_dev.fillna({'type': 'unkown'})

### Save

In [ ]:
card_dev.rename(columns={'type':'card_type', 'disp_id': 'card_disp_id'}, inplace=True)
card_dev.to_csv('data_processed/card_dev.csv', index=False)

## Client preparation

### Extract gender and normalize dates
Since the birth date of each client is represented as YYMMDD when 'Male' or YY(MM+50)DD when 'Female' the gender was extracted as a new feature and the Female client's birth date modified to YYMMDD, the same as Male clients.
Female = False
Male = True

In [ ]:
from datetime import datetime, timedelta

def extract_gender(date):
    month = int(str(date)[2:4])
    gender = ''
    if month > 12:
        month -= 50
        gender = 'Female'
    else: 
        gender = 'Male'
    new_date = str(date)[:2] + str(month).zfill(2) + str(date)[4:]
    return (new_date, gender)

In [ ]:
genders = []
dates = []

for birth_number in client['birth_number']:
    (new_date, gender) = extract_gender(birth_number)
    dates.append(new_date)
    genders.append(gender)

client['birth_number'] = dates
client['gender'] = genders

### Save

In [ ]:
client.rename(columns={'district_id': 'client_district_id'}, inplace=True)
client.to_csv('data_processed/client.csv', index=False)

## Disposition preparation

### Remove disponent clients and add to the account's owner entry
Without this chenage more than one client were associated to a single account_id. This was a problem when merging Dispositions Loans tables since this would lead to several entries with the same loan_id and target classes replicated.

In [ ]:
disp_owners = disposition.query("type == 'OWNER'")
disp_disponent = disposition.query("type == 'DISPONENT'")

has_disponent = [id in disp_disponent['account_id'] for id in disp_owners['account_id']]

disp_owners['has_disponent'] = has_disponent
disp_owners.drop('type', inplace=True, axis=1)
display(disp_owners)

### Save

In [ ]:
disp_owners.to_csv('data_processed/disp.csv', index=False)

## District preparation

### Clean missing values

There was a row with missing values '?' in the district table for both "no. of commited crimes '95" and "unemploymant rate '95" columns. We needed to find a value for it, either replacing it with median or mean value of the column. To decide between median or mean, we need to analyse symmetry and the outliers of each column.

In [ ]:
import matplotlib.pyplot as plt

district_clean = district.loc[(district["unemploymant rate '95"] != '?') & (district["no. of commited crimes '95"] != '?')]


plt.title("Box plot of unemploymant rate '95")
plt.boxplot(district_clean["unemploymant rate '95"].astype('float'))


In [ ]:
plt.title("Box plot of no. of commited crimes '95")
plt.boxplot(district_clean["no. of commited crimes '95"].astype('int'))

By analysing the box plots, we found out that the data values for "no. of commited crimes '95" and "unemploymant rate '95" are not symmetrical and there are outliers, therefore is would be best to use median instead of mean to replace the cells with '?'

In [ ]:

median_unemploymant = district_clean["unemploymant rate '95"].astype(float).median()
median_commited_crimes = district_clean["no. of commited crimes '95"].astype(int).median()


district.loc[district["no. of commited crimes '95"] == '?', "no. of commited crimes '95"] = median_commited_crimes
district.loc[district["unemploymant rate '95"] == '?', "unemploymant rate '95"] = median_unemploymant

# Convert Object Type to Numeric Type
district["unemploymant rate '95"] = pd.to_numeric(district["unemploymant rate '95"])
district["no. of commited crimes '95"] = pd.to_numeric(district["no. of commited crimes '95"])


# Add ratios
# district_df['ratio enterpreneurs'] = district_df['no. of enterpreneurs per 1000 inhabitants'] / 1000
# district_df['ratio of urban inhabitants'] = district_df['ratio of urban inhabitants'] / 100

# district.drop(columns=['name', 'no. of enterpreneurs per 1000 inhabitants', 'no. of inhabitants',
# "unemploymant rate '96", "no. of commited crimes '96",
# "unemploymant rate '95", "no. of commited crimes '95"], inplace=True)

### Save

In [ ]:
# TODO - clean district table
district['code'].to_csv('data_processed/district.csv', index=False)

## Transaction preparation

In [ ]:
# Drop columns with more than 70% of missing values
trans_dev.dropna(thresh=len(trans_dev) * 0.3, axis=1, inplace=True)
trans_dev

### Feature Engineering - Monthly Income and last balance for each account_id

In [ ]:
# Replace type 'withdrawal in cash' with 'withdrawal'
trans_dev.replace({'type': 'withdrawal in cash'}, 'withdrawal', inplace=True)

# Replace NaN of operation with mode for each type
credit_operation_mode = trans_dev[trans_dev['type'] == 'credit']['operation'].mode()[0]
withdrawal_operation_mode = trans_dev[trans_dev['type'] == 'withdrawal']['operation'].mode()[0]
trans_dev['operation'] = trans_dev.apply(lambda x: credit_operation_mode if (x['type'] == 'credit' and pd.isnull(x['operation'])) else (withdrawal_operation_mode if (x['type'] == 'withdrawal' and pd.isnull(x['operation'])) else x['operation']), axis=1)


In [ ]:
trans_dev['date'] = trans_dev['date'].apply(lambda x: '19'+x[:2]+'-'+x[2:4]+'-'+x[4:])
trans_dev['date'] = pd.to_datetime(trans_dev['date'])

In [ ]:
aux = trans_dev.groupby(['account_id']).agg({'date': ['min', 'max']})
aux.columns = ['min_date', 'max_date']
aux = aux.reset_index(drop=False)
aux['date_diff'] = (aux['min_date']-aux['max_date'])/np.timedelta64(1, 'M')
aux['date_diff'] = aux['date_diff'].astype(int)
trans_dev = pd.merge(trans_dev, aux[['account_id', 'date_diff']], on='account_id', how='left')
display(trans_dev)

In [ ]:
def agg_func(data):
    credits = data[(data['type'] == 'credit') & (data['operation'] != 'collection from another bank')]
    withdrawals = data[(data['type'] == 'withdrawal') & (data['operation'] != 'remittance to another bank')]

    return pd.Series({
        'monthly_credit': (credits['amount'].sum() / (abs(data['date_diff'].iat[0]) + 1)) if credits.shape[0] != 0 else 0, 
        'monthly_withdrawal': (withdrawals['amount'].sum() / (abs(data['date_diff'].iat[0]) + 1)) if withdrawals.shape[0] != 0 else 0,
        'last_balance': data['balance'].iat[-1],
    })
trans_agg = trans_dev.groupby(['account_id']).apply(agg_func)
trans_agg = trans_agg.reset_index(drop=False)
trans_agg['monthly_diff'] = (trans_agg['monthly_credit'] - trans_agg['monthly_withdrawal']).astype(int)
trans_agg = trans_agg.drop(['monthly_credit', 'monthly_withdrawal'], axis=1)
trans_agg

### Save

In [ ]:
trans_agg.to_csv('data_processed/trans_dev.csv', index=False)

## Data Integration

In [ ]:
loan_dev = pd.read_csv('data_processed/loan_dev.csv', dtype={'date':object})
account = pd.read_csv('data_processed/account.csv', dtype={'date':object})
card_dev = pd.read_csv('data_processed/card_dev.csv', dtype={'issued':object})
client = pd.read_csv('data_processed/client.csv')
disposition = pd.read_csv('data_processed/disp.csv')
district = pd.read_csv('data_processed/district.csv')
trans_dev = pd.read_csv('data_processed/trans_dev.csv', dtype={'date':object})

pd.set_option('display.max_columns', None)


### Join tables

In [ ]:
data = pd.merge(loan_dev, account, left_on="account_id", right_on="account_id", how='left')
data = pd.merge(data, disp_owners, left_on="account_id", right_on="account_id", how='left')
data = pd.merge(data, client, left_on="client_id", right_on="client_id", how='left')
data = pd.merge(data, card_dev, left_on="disp_id", right_on="card_disp_id", how='left')
data = pd.merge(data, trans_dev, left_on='account_id', right_on='account_id', how='left')
data = pd.merge(data, district, left_on="client_district_id", right_on="code", how='left')
display(data.head())

In [ ]:
# Drop columns with more than 70% of missing values
data.dropna(thresh=len(data) * 0.3, axis=1, inplace=True)
display(data.head())

### Feature Engineering - Age of the client at the loan time

In [ ]:
def get_datetime(date):
    year = int(str(date)[0:2]) + 1900
    month = int(str(date)[2:4])
    day = int(str(date)[4:])
    return datetime(year, month, day)

def calc_age(birth_date, other_date):
    diff:timedelta = get_datetime(other_date) - get_datetime(birth_date)
    return round(diff.days / 365.25)


In [ ]:
data['age_on_loan_request'] = data.apply(lambda x: calc_age(x['birth_number'], x['loan_date']), axis=1)
data.drop('loan_date', inplace=True, axis=1)
data.drop('birth_number', inplace=True, axis=1)
data

### Discretizations

##### Age on loan request

In [ ]:
data['age_on_loan_request_disc'] = pd.cut(x=data['age_on_loan_request'], bins=[0, 19, 29, 39, 49, 59, 69, 79, 99], labels=['0-19', '21-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-99'])
data['age_on_loan_request_disc'] = data['age_on_loan_request_disc'].astype(str)
data

### Save Data

In [ ]:
data.to_csv('data_processed/data.csv', index=False)